In [8]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path
import yfinance as yf 
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from scipy.stats.mstats import winsorize
import os
import glob
import dateutil.parser as dparser


import scipy
%run pyrmt.ipynb #other functions 

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
pyrmt.ipynb:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  "  O. Ledoit and M. Wolf\n",


In [9]:
"""Ledoit & Wolf constant correlation unequal variance shrinkage estimator."""
from typing import Tuple
import numpy as np

def shrinkage(returns: np.array) -> Tuple[np.array, float, float]:
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True)
    returns -= mean_returns
    sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

In [10]:
def shrinkage_EMW(returns_tmp: np.array, lookback = 126) -> Tuple[np.array, float, float]:
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    returns = returns_tmp.tail(lookback).values
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True) # make EWMA
    returns -= mean_returns
    COV_tmp = returns_tmp.ewm(span = lookback).cov()
    idx = returns_tmp.index.get_level_values(0)[-1]
    sample_cov = COV_tmp[COV_tmp.index.get_level_values(0) == idx]
    sample_cov = sample_cov.values
    #sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

In [11]:
from scipy.stats import norm
import ezodf
import scipy.optimize as sco
import scipy

from sklearn.covariance import LedoitWolf

def Optimize_Portfolio(data ,lookback = 126, risk_free = 0, objective = 'Kelly'):

    ret = (data-1).mean()
    #cov_fit = LedoitWolf().fit(data)
    #cov = cov_fit.covariance_
    cov, average_cor, shrink = shrinkage_EMW(data, lookback = lookback)
    #cov = PCA_cov(data, N=5)
   
  
    if objective == 'Max Div':
        num_assets = len(data.columns)
        args = (cov)
        constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  #{'type':'ineq', 'fun': lambda x: 1 - np.sum(x)} # sum <= 1
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
        
        result = sco.minimize(calc_diversification_ratio, num_assets*[1./num_assets,], args=args, 
                              method='SLSQP', constraints=constraints, tol = 0.0000000000000000000000001)
        
    elif objective == "min var":
        num_assets = len(data.columns)
        args = (cov)
        constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  #{'type':'ineq', 'fun': lambda x: 1 - np.sum(x)} # sum <= 1
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
        
        result = sco.minimize(port_var, num_assets*[1./num_assets,], args=args, 
                              method='SLSQP', constraints=constraints, tol = 0.0000000000000000000000001)
    elif objective == "erc":
        num_assets = len(data.columns) 
        args = (cov)
        constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  #{'type':'ineq', 'fun': lambda x: 1 - np.sum(x)} # sum <= 1
                  {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                      {'type':'ineq', 'fun': lambda x: x-(1/num_assets)*0.7}, # min position
                      {'type':'ineq', 'fun': lambda x: (1/num_assets)*1.3-x}) # max position
        
        result = sco.minimize(erc, num_assets*[1./num_assets,], args=args, 
                              method='SLSQP', constraints=constraints, tol = 0.0000000000000000000000001)
        

    return (result)




def port_var(weights, cov):
    var = weights.dot(cov).dot(weights)
    return(var)

def port_ret(weights, ret, risk_free = 0):
    #needs to be array
    ret = ret - risk_free
    port_ret = weights.dot(ret)
    return(port_ret)

def risk_parity(data):
    vol = np.log((data)).std()

    sum_vol = 0
    for i in range(len(vol)):
        sum_vol =sum_vol + (1/vol[i])
    
    weight = []
    for i in range(len(vol)):
        w = (1/vol[i])/(sum_vol)
        weight.append(w)
   
    weight = [round(num, 2) for num in weight]
    return(weight)




def calc_diversification_ratio(weights, cov):
    # average weighted vol
    w_vol = np.dot(np.sqrt(np.diag(cov)), weights.T)
    # portfolio vol
    port_vol = np.sqrt(port_var(weights, cov))
    
    diversification_ratio = w_vol/port_vol
    # return negative for minimization problem (maximize = minimize -)
    return -diversification_ratio

def erc(weights, cov):
        # these are non normalized risk contributions, i.e. not regularized
        # by total risk, seems to help numerically
        risk_contributions = np.dot(weights, cov) * weights
        a = np.reshape(risk_contributions, (len(risk_contributions), 1))
        # broadcasts so you get pairwise differences in risk contributions
        risk_diffs = a - a.transpose()
        sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
        # https://stackoverflow.com/a/36685019/1451311
        return sum_risk_diffs_squared #/ scale_factorcov
    


import sklearn.datasets, sklearn.decomposition

def PCA_cov(data, N = 5):
    
    X = data.ewm(span = 252).cov()
    DATE_IDX = X.index.get_level_values(level=0)[-1]
    X = X[X.index.get_level_values(0)==DATE_IDX].droplevel(0)
    mu = np.mean(X, axis=0)

    pca = sklearn.decomposition.PCA()
    pca.fit(X)

    nComp = N
    Xhat = np.dot(pca.transform(X)[:,:nComp], pca.components_[:nComp,:])
    Xhat += mu
    clean_cov = pd.DataFrame(Xhat)
    clean_cov.index = X.index
    clean_cov.columns = X.index
    return(clean_cov)

In [12]:
def ERC_gestalt(data, lookback = 126):
    
    prices_df = pd.DataFrame()
    for tick in data['Yahoo']:
    
        price = yf.download(tick,start='2000-01-01', progress = False, threads = False)
        price = price['Adj Close']
        prices_df[tick] = price
    
    log_ret = np.log(prices_df) - np.log(prices_df.shift(1))
    log_ret = log_ret.dropna()
    weight = Optimize_Portfolio(log_ret, lookback = lookback, objective='erc')['x'].round(3)

    return(weight)

def round_to_multiple(number, multiple):
    return multiple * round(number / multiple)

In [13]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import inv,pinv
from scipy.optimize import minimize

 # risk budgeting optimization
def calculate_portfolio_var(w,V):
    # function that calculates portfolio risk
    w = np.matrix(w)
    return (w*V*w.T)[0,0]

def calculate_risk_contribution(w,V):
    # function that calculates asset contribution to total risk
    w = np.matrix(w)
    sigma = np.sqrt(calculate_portfolio_var(w,V))
    # Marginal Risk Contribution
    MRC = V*w.T
    # Risk Contribution
    RC = np.multiply(MRC,w.T)/sigma
    return RC

def risk_budget_objective(x,pars):
    # calculate portfolio risk
    V = pars[0]# covariance table
    x_t = pars[1] # risk target in percent of portfolio risk
    sig_p =  np.sqrt(calculate_portfolio_var(x,V)) # portfolio sigma
    risk_target = np.asmatrix(np.multiply(sig_p,x_t))
    asset_RC = calculate_risk_contribution(x,V)
    J = sum(np.square(asset_RC-risk_target.T))[0,0] *10000000 # sum of squared error
    return J

def total_weight_constraint(x):
    return np.sum(x)-1.0

def long_only_constraint(x):
    return x

## Import old portfolios to construct staggerd portfolio

- Q: How to handel "hold" positions?
- "Hold" companies shold have "Min Position" == ACTION/3 rest is weighted from this? and MAX = Average posiotn

In [14]:
### Settings
START_DATE = '2016-01-01'
error_count = 0
error_list = []


latest_file= max(glob.glob("../equity_data/*.*"), key=os.path.getmtime)

signal_df = pd.read_excel(latest_file, sheet_name="Export")
signal_df = signal_df.rename({'Performance - Perform. 3m' : 'Return 3m','Performance - Perform. 6m' : 'Return 6m',
                            'Total Return - Return 1y' : 'Return 1y',
                            'Div. Yield - Current': 'Yield',
                            'Total Equity  - Millions':'Total Equity', 'FCF - Millions': 'FCF','ROE - Current':'ROE',
                            'Volatility - St.Dev. 100d':'Volatility','Market Cap - Current': 'Market Cap', 
                            'ROC - Current':'ROC', 'Tot. Assets - Millions':'Tot. Assets', 
                            'Gross profit - Millions':'Gross profit', 'Assets Turn - Current': 'Assets Turn',
                            'P/FCF - Current':'P/FCF', 'P/E - Current':'P/E', 'P/S - Current':'P/S',
                            'P/B - Current':'P/B','EV/EBIT - Current':'EV/EBIT',
                            'Info - Country' : 'Country','F-Score - Point':'F-Score',
                            'Info - List' : 'List', 'Info - Sector' : 'Sector', 'Info - Industry' : 'Industry',
                            'Info - Ticker' : 'Ticker', 'Info - Yahoo':'Yahoo', 'Info - Last Report': 'Last Report',
                           'Volume - Average 50d Mill' : 'Volume', 'Tot. Assets - Growth 1y' : 'Asset Growth'}, axis=1)


signal_df = signal_df.loc[ (signal_df['List'] != 'Spotlight') 
                        & (signal_df['List'] != 'NGM') & (signal_df['Country'] == "Sweden") &
                         (signal_df['Market Cap'] > 200)]

signal_df = signal_df.loc[(signal_df['Sector'] != 'Financials')]

# Set to dattime
signal_df['Last Report'] = pd.to_datetime(signal_df['Last Report'])
#set new index
signal_df.index = range(len(signal_df.index))


signal_df['Res_Mom_1M'] = np.nan
signal_df['Res_Mom_1M_alt'] = np.nan

signal_df['Tot_Mom_1M'] = np.nan
signal_df['Sea_month_5yr'] = np.nan
signal_df['idio_vol_20day'] = np.nan
signal_df['maxret_5days'] = np.nan
signal_df["EAR_std"]= np.nan
signal_df["5yr_vol"]= np.nan
signal_df["liq_shock"]= np.nan

index = yf.download('^OMXSPI',start=START_DATE, threads = False, progress = False)
index = index['Adj Close']
for i in tqdm(range(len(signal_df))):

    try:
        stock_tmp = yf.download(signal_df.iloc[i]['Yahoo'],start=START_DATE, progress = False, threads = False)

        
        stock = stock_tmp['Adj Close']
        import_data = pd.concat([stock, index], axis = 1)
        import_data.columns = ['stock', 'index']
        import_data = import_data.dropna()
        
        long_df = import_data.copy()
        ret_df = np.log(import_data/import_data.shift()).dropna()
        ### SEASONALITY
        
        monthly_df = import_data.resample('M').last()
        monthly_ret_df = np.log(monthly_df/monthly_df.shift()).dropna()
         
        ### 1 Month Momentum
        signal_df.loc[i,"Tot_Mom_1M"] = ret_df['stock'].tail(21).sum()
        
        
        ##EAR
        idx = ret_df.index.get_loc(signal_df.iloc[i]['Last Report'], method='nearest')
        
        EA_data = import_data.iloc[idx - 2 : idx +2 ]
        
        EA_ret = (EA_data.pct_change().dropna()+1).cumprod().tail(1)
        pead_ret = float(EA_ret['stock'] - EA_ret['index']) #Should use np.log()
        pead_vol = np.log(stock.iloc[:idx]/stock.iloc[:idx].shift()).tail(60).std()*252**.5
        signal_df.loc[i, 'EAR_std'] = pead_ret/pead_vol
        
        ## liquidity shock
        
        stock_volume = stock_tmp.copy()
        stock_volume['volume_sek'] = stock_volume['Close'] *stock_volume['Volume']
        
        # Resample to monthly for sobustness???
        stock_volume = stock_volume.rolling(21).sum().resample('30D').last()
        

        liq_shock = (stock_volume['volume_sek'].tail(1) - stock_volume['volume_sek'].tail(12).mean())/stock_volume['volume_sek'].tail(12).std()
        signal_df.loc[i,"liq_shock"] = float(liq_shock)
        
        ### RESIDUAL MOMENTUM
        ret_trim_df = ret_df.drop(ret_df.iloc[[idx - 1, idx, idx +1, idx +2 ]].index)
        
        
        signal_df.loc[i,"maxret_5days"] = np.mean(sorted(ret_trim_df['stock'].tail(21))[-5:])
        y_res = ret_trim_df.tail(21)['stock']
        X_res = np.array(ret_trim_df.tail(21)['index']).reshape(-1, 1)
        reg_res = LinearRegression().fit(X_res, y_res)
        residuals_res = y_res - reg_res.predict(X_res)
        signal_df.loc[i,"idio_vol_20day"] = residuals_res.std()
        
        #volume weight for short term rev
        vol_weight_tmp = stock_tmp.copy()
        vol_weight_tmp['volume_sek'] = vol_weight_tmp['Close'] *vol_weight_tmp['Volume']
        #weight by 60 day MA
        vol_weight = vol_weight_tmp['volume_sek'].rolling(252).mean() / vol_weight_tmp['volume_sek']
        
        reg_df = ret_trim_df.tail(3*252)
        ## Identify Report 
        if len(reg_df)>(2*252):
            y = reg_df['stock']
            X = np.array(reg_df['index']).reshape(-1, 1)
            reg = LinearRegression().fit(X, y)
            beta = reg.coef_[0]
            residuals = y - reg.predict(X)
            std_residuals = residuals/residuals.std()
            
            signal_df.loc[i,"Res_Mom_1M"] = std_residuals.tail(21).sum()# + np.log(vol_weight.tail(21)).sum()
           
        
        if len(monthly_ret_df)>=36:
            seas_list = []
            monthly_vol = 0
            for look in [12,24,36,48,60]:
                try:
                    seas_list.append(monthly_ret_df['stock'].iloc[-look])
                    monthly_vol = (monthly_ret_df['stock'].tail(60)+1).std() * np.sqrt(12)
                except:
                    pass
        
            signal_df.loc[i,"Sea_month_5yr"] = np.mean(seas_list) *12
            signal_df.loc[i,"5yr_vol"] = monthly_vol
            
            
    except:
        error_count = error_count + 1
        error_list.append(i)

 69%|██████▉   | 332/482 [01:55<00:47,  3.19it/s]


1 Failed download:
- SDIP.ST: No data found, symbol may be delisted


100%|██████████| 482/482 [02:54<00:00,  2.77it/s]


In [15]:
latest_file

'../equity_data/Borsdata_2022-08-27.xlsx'

In [44]:
#### CREATE NEW SECTORS!
signal_df.loc[signal_df['Industry'].isin(
    ['Leisure', 'Gambling & Casinos','Airlines','Hotels']),'Sector'] = 'Travel & Leisure'


signal_df.loc[signal_df['Industry'].isin(
    ['Pharmaceuticals']),'Sector'] = 'Pharmaceuticals'

signal_df.loc[signal_df['Industry'].isin(
    ['Medical Equipment']),'Sector'] = 'Medical Equipment'


signal_df.loc[signal_df['Industry'].isin(
    ['Retailers','Auto & Equipment','Industrial Components', 'Clothing & Footwear',
    'Consumer Electronics', 'Accessories' ]),'Sector'] = 'Retail'


signal_df.loc[signal_df['Industry'].isin(
    ['IT Consulting', 'IT Services', 'Communications',]),'Sector'] = 'Software'



signal_df.loc[signal_df['Industry'].isin(
    ['Industrial Components',
    'Energy & Recycling' ]),'Sector'] = 'General Industrials'



signal_df.loc[signal_df['Industry'].isin(
    ['Construction Supplies','Construction & Infrastructure',
     'Installation']),'Sector'] = 'Construction & Materials' 



signal_df.loc[signal_df['Industry'].isin(
    ['Industrial Machinery', 'Electrical Components']),'Sector'] = 'Electronic & Electrical Equipment'

signal_df.loc[signal_df['Industry'].isin(
    ['Holding Companies']),'Sector'] = 'Holding Companies'


signal_df.loc[signal_df['Industry'].isin(
    ['Real Estate']),'Sector'] = 'Real Estate'


In [45]:
CUTOFF = 0.33 #0.25 # which cut off??

method = 'median'

### SHORT TERM REVERSAL - adjust for industry ###
signal_df['Res_Mom_1M_adj'] = signal_df["Res_Mom_1M"] - signal_df.groupby("Sector")["Res_Mom_1M"].transform(method)


#Vol adjsuted Seasonality
signal_df['Sea_month_5yr_std'] = signal_df['Sea_month_5yr']/signal_df['5yr_vol']

### IVOL - adjust for industry
signal_df['idio_vol_20day_adj'] = signal_df["idio_vol_20day"] - signal_df.groupby("Sector")["idio_vol_20day"].transform(method)

## MAX RET - adjust for industry
signal_df['maxret_5days_adj'] = signal_df["maxret_5days"] - signal_df.groupby("Sector")["maxret_5days"].transform(method)

########## INDUSTRY MOMENTUM ASSNESS SHOWS THAT EQUAL WEIGHT WORKS
signal_df['Sector Weighted Mom'] = signal_df.groupby("Sector")["Tot_Mom_1M"].transform(method)

# IMPUTE MEDIAN VALUE FOR NANS
signal_df['Res_Mom_1M_adj'] = signal_df['Res_Mom_1M_adj'].fillna(signal_df['Res_Mom_1M'].median())

### SEASONALITY
signal_df['Sea_month_5yr_std'] = signal_df['Sea_month_5yr_std'].fillna(signal_df['Sea_month_5yr_std'].median())
### SHORT TERM IVOL
signal_df['idio_vol_20day_adj'] = signal_df['idio_vol_20day_adj'].fillna(signal_df['idio_vol_20day'].median())
### MAX RET
signal_df['maxret_5days_adj'] = signal_df['maxret_5days_adj'].fillna(signal_df['maxret_5days_adj'].median())
# Ear 
signal_df['EAR_std'] = signal_df['EAR_std'].fillna(signal_df['EAR_std'].median())
# Liquidity Shock
signal_df['liq_shock'] = signal_df['liq_shock'].fillna(signal_df['liq_shock'].median())



######### RANK ON INDIVIDUAL PREDICTORS
signal_df['Res_Mom_1M_adj_rank'] = signal_df['Res_Mom_1M_adj'].rank(ascending=True, pct = True)
#high is good
signal_df['Sector Momentum Rank'] =  signal_df['Sector Weighted Mom'].rank(ascending=False, pct = True)
#high is good
signal_df['Seasonality Rank'] =  signal_df['Sea_month_5yr'].rank(ascending=False, pct = True)
#high is good
signal_df['Seasonality Rank Std'] =  signal_df['Sea_month_5yr_std'].rank(ascending=False, pct = True)
#low is good
signal_df['IVOL_adj Rank'] =  signal_df['idio_vol_20day_adj'].rank(ascending=True, pct = True)
#low is good
signal_df['MAXRET_adj Rank'] =  signal_df['maxret_5days_adj'].rank(ascending=True, pct = True)
#high is good
signal_df['EAR_std Rank'] =  signal_df['EAR_std'].rank(ascending=False, pct = True)
#high is good
signal_df['liq_shock Rank'] =  signal_df['liq_shock'].rank(ascending=False, pct = True)

################# YOU WANT THE LOWEST SCORE POSSIBLE
## USE PCT. 
## Implement an interaction score for liquidity and SREV 

signal_df['High_Freq_Combo'] = ( signal_df['Sector Momentum Rank'] +
                               signal_df['Res_Mom_1M_adj_rank'] +
                               signal_df['Seasonality Rank Std'] +  signal_df['EAR_std Rank'] +
                                signal_df['liq_shock Rank']+
                                0.5*signal_df['IVOL_adj Rank'] + 0.5*signal_df['MAXRET_adj Rank']
                                ).rank(ascending=True)



signal_df['Signal'] = "Neutral"
idx_BUY = signal_df['High_Freq_Combo']<=signal_df['High_Freq_Combo'].quantile(CUTOFF)
signal_df.loc[idx_BUY,'Signal']= 'Buy'


idx_SELL = signal_df['High_Freq_Combo']>=signal_df['High_Freq_Combo'].quantile(1-CUTOFF)
signal_df.loc[idx_SELL,'Signal'] = 'Sell'
rank_data = signal_df



### OPTIMIZE PORTFOLIO

In [46]:
def get_hvol_yz(df, lookback=10):
    o = df.Open
    h = df.High
    l = df.Low
    c = df.Close # should this be 
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    #close_vol = pd.rolling_sum(cc_sq, window=lookback) * (1.0 / (lookback - 1.0))
    close_vol =  cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol =  oc_sq.rolling(lookback).sum()  * (1.0 / (lookback - 1.0))
    window_rs =  rs.rolling(lookback).sum()  * (1.0 / (lookback - 1.0))
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) 
    result[:lookback-1] = np.nan
    return result


In [47]:

#### IMPORt DATA AND CREATE MA of SIGNAL

SPREAD = 3
N = 15


folder = "../clean_equity_data/"

port_file = ("../portfolios/eriks_port.xlsx")


current_port_tmp = pd.read_excel(port_file)
current_cash = current_port_tmp.loc[current_port_tmp['Company'].isin(["Cash"])]
current_port_tmp = current_port_tmp.loc[current_port_tmp['Current %']>0,]
current_port_tmp = current_port_tmp.loc[~current_port_tmp['Company'].isin(["Cash", "Total"])]
current_port = current_port_tmp[['Company','Current %' ]]
current_port = current_port[~current_port['Company'].isna()]

file_list = ["GESTALT_2022-08-27.csv","GESTALT_2022-08-27.csv", "GESTALT_2022-08-27.csv" ]
#file_list = ["GESTALT_2022-06-29.csv","GESTALT_2022-07-28.csv", "GESTALT_2022-08-27.csv" ]


In [48]:


for idx, file in enumerate(file_list):
    data_tmp = pd.read_csv(folder + file)
    data_tmp.loc[:, 'Signal_' + str(idx)] =  np.linspace(1, -1, num=len(data_tmp))
    data_tmp_clean = data_tmp[['Company','Yahoo', 'Signal_' + str(idx)]]
    if idx == 0:
        data_comb = data_tmp_clean
    else:
        data_comb = data_comb.merge(data_tmp_clean, how ='outer', on = ['Company','Yahoo'] )
    
    
#Fill missing values with 0
data_comb[data_comb.filter(like='Signal').columns] = data_comb[data_comb.filter(like='Signal').columns].fillna(value=0)

### Get avergae signal value over the columns
data_comb['Signal_avg'] = data_comb[data_comb.filter(like='Signal').columns].mean(axis = 1)
sig_scaled = (2*data_comb['Signal_avg'])/ abs(data_comb['Signal_avg']).sum() #scale as aqr
data_comb['Signal_avg'] = sig_scaled

data_comb = data_comb.sort_values(by = 'Signal_avg', ascending=False)


In [49]:
##### TOP STOCKS, GET 

top = data_comb[0:SPREAD*N][['Company','Yahoo', 'Signal_avg']] 
top.loc[:,'assets_risk_budget'] = np.linspace(1, 0.5, num=SPREAD*N)

##### SELECT THE PORTFOLIO BY BUY/HOLD SPREAD

buy = top[0:N][['Company','Yahoo' ,'Signal_avg', 'assets_risk_budget']]
hold = top[N: round(SPREAD*N)][['Company','Yahoo', 'Signal_avg', 'assets_risk_budget']] # update to latest month spread??
keep = hold[hold['Company'].isin(current_port['Company'])]
    
input_opti = pd.concat([buy,keep])

## RESCALE ASSETS_RISK_BUDGET

rescaled_risk_budget = input_opti['assets_risk_budget']/input_opti['assets_risk_budget'].sum()
input_opti['adj_risk_budget'] = rescaled_risk_budget

SELL_LIST =pd.DataFrame(list(set(current_port['Company']) - set(input_opti['Company'])),columns = ['Company'] )
#SELL_LIST = SELL_LIST.groupby(['Company']).sum()
SELL_LIST = SELL_LIST.merge(rank_data[['Company','Signal', "Ticker", "Yahoo"]], on = 'Company')



# EXPERIMENT
https://thequantmba.wordpress.com/2016/12/14/risk-parityrisk-budgeting-portfolio-in-python/

In [52]:
def constr_f(x):
    return np.array(sum(x))


def ERC_gestalt_alt(data, lookback = 252):
    
    prices_df = pd.DataFrame()
    vol_yz = pd.DataFrame()
    for tick in data['Yahoo']:
    
        price_imp_tmp = yf.download(tick,start='2000-01-01', progress = False, threads = False)
        price = price_imp_tmp['Adj Close']
        price_tmp = pd.DataFrame(price)
        price_tmp.columns = [tick]
        prices_df = pd.concat([prices_df, price_tmp], axis = 1)
        #append to array instead
        vol_yz[tick] = get_hvol_yz(price_imp_tmp,lookback = lookback).tail(1)



    vol_yz_ls = vol_yz.values
    vol_yz_ls[np.isnan(vol_yz_ls)] = np.nanmean(vol_yz_ls)


    log_ret = np.log(prices_df) - np.log(prices_df.shift(1))
    log_ret = log_ret.tail(300)
    log_ret = log_ret.fillna(log_ret.mean())


    #cov = pyRMT.optimalShrinkage(log_ret.tail(252) , return_covariance=True) #we want the regurlized
    corr = optimalShrinkage(log_ret.tail(lookback), method = 'IW') #regurlized
    cov = vol_yz_ls.T * corr * vol_yz_ls   
        
        
    log_ret = np.log(prices_df) - np.log(prices_df.shift(1))
    log_ret = log_ret.dropna()
    
    
     
    bounds = [(0.01,0.3)] *len(input_opti)


    # sum of weights beteen 0.99 and 1.01
    nlc = optimize.NonlinearConstraint(constr_f, 0.99, 1.01)
    
    x_t =data['adj_risk_budget']# your risk budget percent of total portfolio risk (equal risk)


    
    # Add constrinarts?
    res_diff = optimize.differential_evolution(risk_budget_objective, args=[[cov,x_t]],
                                bounds = bounds, maxiter = 10000,
                                 constraints = nlc          )
    print(res_diff)
    weight = res_diff['x']/res_diff['x'].sum()

    return(weight, cov)





In [53]:
import scipy.optimize as optimize
res_w, cov = ERC_gestalt_alt(input_opti)

In [90]:


new_input_opti = input_opti.copy()
new_input_opti['new_weight'] = np.array(round_to_multiple(pd.DataFrame(res_w), 0.001))
new_input_opti['vol'] = np.sqrt(np.diagonal(cov)) * np.sqrt(252)
new_input_opti['risk_budget'] = input_opti['adj_risk_budget']
new_input_opti['risk_contrib'] = calculate_risk_contribution(res_w,cov)/calculate_risk_contribution(res_w,cov).sum()

new_port = new_input_opti[['Company','Yahoo','Signal_avg','new_weight', 'adj_risk_budget', 'risk_contrib']]



In [91]:
final_port = new_port.merge(rank_data[['Company','Signal', "Ticker"]], on = 'Company')

new_port = pd.concat([final_port,SELL_LIST]).fillna(0)


In [92]:
new_port

,Company,Yahoo,Signal_avg,new_weight,adj_risk_budget,risk_contrib,Signal,Ticker
0,Arctic Paper,ARP.ST,0.013245,0.046,0.056884,0.056884,Buy,ARP
1,Dedicare,DEDI.ST,0.013157,0.055,0.056238,0.056238,Buy,DEDI
2,Rottneros,RROS.ST,0.013068,0.058,0.055591,0.055591,Buy,RROS
3,International Petroleum,IPCO.ST,0.012980,0.042,0.054945,0.054945,Buy,IPCO
4,SSAB B,SSAB-B.ST,0.012892,0.055,0.054299,0.054299,Neutral,SSAB B
5,EnQuest,ENQ.ST,0.012804,0.039,0.053652,0.053652,Buy,ENQ
6,B3 Consulting,B3.ST,0.012715,0.043,0.053006,0.053006,Neutral,B3
7,New Wave,NEWA-B.ST,0.012627,0.045,0.052359,0.052359,Neutral,NEWA B
8,Betsson,BETS-B.ST,0.012539,0.061,0.051713,0.051713,Buy,BETS B
9,Prevas,PREV-B.ST,0.012450,0.047,0.051067,0.051067,Sell,PREV B


In [62]:
prices_df = pd.DataFrame()

for tick in new_port['Yahoo']:
    if tick == 'Cash':
        prices_df[tick] = 10
    else:
        price = yf.download(tick,start='2000-01-01', progress = False, threads = False)
        price = price['Adj Close']
        prices_df[tick] = price
    
log_ret = np.log(prices_df) - np.log(prices_df.shift(1))
log_ret = log_ret.dropna() 

hist_port = log_ret.mul(new_port['new_weight'].values, axis=1).sum(axis = 1).tail(252)

prices_df = pd.DataFrame()
for tick in ["^OMX"]:
    
    price = yf.download(tick,start='2000-01-01', progress = False, threads = False)
    price = price['Adj Close']
    prices_df[tick] = price
log_ret_tmp = np.log(prices_df) - np.log(prices_df.shift(1))
log_ret_tmp = log_ret_tmp.dropna()

from sklearn.linear_model import LinearRegression

y = hist_port
X = np.array(log_ret_tmp.tail(252))#.reshape(-1, 1)
reg = LinearRegression().fit(X, y)

In [63]:
date_signal = pd.DataFrame([dparser.parse(latest_file,fuzzy=True).strftime("%d/%m/%Y")])
date_signal.columns = ["Date"]

beta = pd.DataFrame([reg.coef_])
beta.columns = ["Beta"]
vol = pd.DataFrame([hist_port.tail(60).std() * np.sqrt(252)])
vol.columns = ["H-Vol"]

In [65]:
max_positions = SPREAD * N

clean_write = pd.DataFrame([])

Zeros =  [0] * max_positions
Blanks = [" "] * max_positions

clean_write.loc[:,'Antal'] = Zeros
clean_write.loc[:,'Weight'] = Zeros
clean_write.loc[:,'Company'] = Blanks
clean_write.loc[:,'Ticker'] = Blanks
clean_write.loc[:,'Signal'] = Blanks

In [93]:
#rename columns
new_port = new_port.rename({ 'new_weight': 'Weight'}, axis=1)

#add antal
new_port = new_port.merge(current_port_tmp[['Company','Antal' ]], on = 'Company', how = 'outer')
new_port.loc[new_port['Antal'].isna(),'Antal'] = 0
new_port.loc[new_port['Weight'].isna(),'Weight'] = 0

#new_port = new_port.merge(signal_df[['Company','Ticker' ]], on = 'Company', how = 'left')



In [95]:
#remove cash and insert seperatly

new_cash = new_port.loc[new_port['Company'].isin(["Cash"])]
new_port = new_port.loc[~new_port['Company'].isin(["Cash", "Total"])]


In [97]:
#### Write to excel file
#current_port_tmp

from openpyxl import load_workbook
book = load_workbook(port_file)
writer = pd.ExcelWriter(port_file, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

clean_write
## CLEAN OLD FILE
clean_write.to_excel(writer, "Gestalt",columns=['Company'], index = False, startcol = 1)
clean_write.to_excel(writer, "Gestalt",columns=['Ticker'], index = False, startcol = 2)
clean_write.to_excel(writer, "Gestalt",columns=['Antal'], index = False, startcol = 3)
clean_write.to_excel(writer, "Gestalt",columns=['Weight'], index = False, startcol = 4)
clean_write.to_excel(writer, "Gestalt",columns=['Signal'], index = False, startcol = 10)


## WRITE NEW PORTFOLIO TO FILE
new_port.to_excel(writer, "Gestalt",columns=['Company'], index = False, startcol = 1)
new_port.to_excel(writer, "Gestalt",columns=['Ticker'], index = False, startcol = 2)
new_port.to_excel(writer, "Gestalt",columns=['Antal'], index = False, startcol = 3)
new_port.to_excel(writer, "Gestalt",columns=['Weight'], index = False, startcol = 4)
new_port.to_excel(writer, "Gestalt",columns=['Signal'], index = False, startcol = 10)


date_signal.to_excel(writer, "Gestalt", index = False, startcol = 9, startrow=max_positions +1)
beta.to_excel(writer, "Gestalt", index = False, startcol = 10, startrow=max_positions +1)
vol.to_excel(writer, "Gestalt", index = False, startcol = 11, startrow=max_positions +1)



new_cash.to_excel(writer, "Gestalt",columns=['Weight'], index = False, header = False, startcol = 4, startrow=max_positions +1)



writer.save()

